In [59]:
def load_data_from_huggingface(path,name=None,split=None, max_len=20):
    #langchain has a wrapper drops the splits 
    from datasets import load_dataset

    #dataset_url = "hotpot_qa"
    #split = "validation"
    ds = load_dataset(path=path, name=name, split=split).select(range(0, max_len))

In [61]:
load_data_from_huggingface('hotpot_qa',name='distractor',split=SPLIT)

Found cached dataset hotpot_qa (/Users/deanchanter/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)


In [1]:
import pandas as pd

In [5]:
from datasets import load_dataset

DATASET_URL = "hotpot_qa"
SPLIT = "validation"
ds = load_dataset(DATASET_URL, "distractor", split=SPLIT)
ds

Found cached dataset hotpot_qa (/Users/deanchanter/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)


Dataset({
    features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
    num_rows: 7405
})

In [8]:
from datasets import load_dataset

dataset_url = "pubmed_qa"
split = "train"
subset = "pqa_labeled"
ds_raw = load_dataset(dataset_url, subset, split=split)
ds_raw

Found cached dataset pubmed_qa (/Users/deanchanter/.cache/huggingface/datasets/pubmed_qa/pqa_labeled/1.0.0/dd4c39f031a958c7e782595fa4dd1b1330484e8bbadd4d9212e5046f27e68924)


Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 1000
})

In [9]:
ds_raw[1]

{'pubid': 16418930,
 'question': 'Landolt C and snellen e acuity: differences in strabismus amblyopia?',
 'context': {'contexts': ['Assessment of visual acuity depends on the optotypes used for measurement. The ability to recognize different optotypes differs even if their critical details appear under the same visual angle. Since optotypes are evaluated on individuals with good visual acuity and without eye disorders, differences in the lower visual acuity range cannot be excluded. In this study, visual acuity measured with the Snellen E was compared to the Landolt C acuity.',
   '100 patients (age 8 - 90 years, median 60.5 years) with various eye disorders, among them 39 with amblyopia due to strabismus, and 13 healthy volunteers were tested. Charts with the Snellen E and the Landolt C (Precision Vision) which mimic the ETDRS charts were used to assess visual acuity. Three out of 5 optotypes per line had to be correctly identified, while wrong answers were monitored. In the group of 

In [7]:
ds[100]

{'pubid': 22564465,
 'question': 'Mammographic screening in Sami speaking municipalities and a control group. Are early outcome measures influenced by ethnicity?',
 'context': {'contexts': ['Female citizens of Sami (the indigenous people of Norway) municipalities in northern Norway have a low risk of breast cancer. The objective of this study was to describe the attendance rate and outcome of the Norwegian Breast Cancer Screening Program (NBCSP) in the Sami-speaking municipalities and a control group.',
   'A retrospective registry-based study.',
   'The 8 municipalities included in the administration area of the Sami language law (Sami) were matched with a control group of 11 municipalities (non-Sami). Population data were accessed from Statistics Norway. Data regarding invitations and outcome in the NBCSP during the period 2001-2010 was derived from the Cancer Registry of Norway (CRN). The NBCSP targets women aged 50-69 years. Rates and percentages were compared using chi-square test

In [12]:
df_raw = ds_raw.to_pandas()

In [16]:
pd.set_option('display.max_colwidth', None)
df_raw.iloc[0]

pubid                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [19]:
def extract_columns(row):
    row['context_str'] = ','.join(row['context']['contexts'])
    row['labels'] = ', '.join(row['context']['labels'])
    row['meshes'] = ', '.join(row['context']['meshes'])
    return row

df_clean = df_raw.apply(extract_columns, axis=1).drop(columns=['context'])

df_clean

,pubid,question,long_answer,final_decision,context_str,labels,meshes
0,21645374,Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?,"Results depicted mitochondrial dynamics in vivo as PCD progresses within the lace plant, and highlight the correlation of this organelle with other organelles during developmental PCD. To the best of our knowledge, this is the first report of mitochondria and chloroplasts moving on transvacuolar strands to form a ring structure surrounding the nucleus during developmental PCD. Also, for the first time, we have shown the feasibility for the use of CsA in a whole plant system. Overall, our findings implicate the mitochondria as playing a critical and early role in developmentally regulated PCD in the lace plant.",yes,"Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.,The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). Window stage leaves were stained with the mitochondrial dye MitoTracker Red CMXRos and examined. Mitochondrial dynamics were delineated into four categories (M1-M4) based on characteristics including distribution, motility, and membrane potential (ΔΨm). A TUNEL assay showed fragmented nDNA in a gradient over these mitochondrial stages. Chloroplasts and transvacuolar strands were also examined using live cell imaging. The possible importance of mitochondrial permeability transition pore (PTP) formation during PCD was indirectly examined via in vivo cyclosporine A (CsA) treatment. This treatment resulted in lace plant leaves with a significantly lower number of perforations compared to controls, and that displayed mitochondrial dynamics similar to that of non-PCD cells.","BACKGROUND, RESULTS","Alismataceae, Apoptosis, Cell Differentiation, Mitochondria, Plant Leaves"
1,16418930,Landolt C and snellen e acuity: differences in strabismus amblyopia?,"Using the charts described, there was only a slight overestimation of visual acuity by the Snellen E compared to the Landolt C, even in strabismus amblyopia. Small differences in the lower visual acuity range have to be considered.",no,"Assessment of visual acuity depends on the optotypes used for measurement. The ability to recognize different optotypes differs even if their critical details appear under the same visual angle. Since optotypes are evaluated on individuals with good visual acuity and without eye disorders, differences in the lower visual acuity range cannot be excluded. In this study, visual acuity measured with the Snellen E was compared to the Landolt C acuity.,100 patients (age 8 - 90 years, median 60.5 years) with various eye disorders, among them 39 with amblyopia due to strabismus, and 13 healthy volunteers were tested. Charts with the Snellen E and the Landolt C (Precision Vision) which mimic the ETDRS charts were used to assess visual acuity. Three out of 5 optotypes per line had to be correctly identified, while wrong answers were monitored. In the group of patients, the eyes with the lower visual acuity, and the right eyes of the healthy subjects, were evaluated.,Differences between Landolt C acuity (LR) and Snellen E acuity (SE) were small. The mean decimal values for LR and SE were

In [21]:
from langchain.document_loaders import DataFrameLoader
loader = DataFrameLoader(df_clean, page_content_column="context_str")
docs  = loader.load()

In [22]:
docs[0]

Document(page_content='Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.,The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). Window stage leaves were stained with the mitochondrial dye Mi